# E(n)-Equivariant Steerable CNNs  -  A concrete example


In [1]:
try:
    import setGPU
except:
    pass
    
import inspect
import sys
sys.path.append('../')

import torch

import jax
import jax.numpy as jnp
import equinox as eqx
import optax
from jaxtyping import Array, Float, Int, PyTree, PRNGKeyArray

from escnn import gspaces
from escnn import nn

setGPU: Setting GPU to: 1


Finally, we build a **Steerable CNN** and try it on MNIST.

Let's also use a group a bit larger: we now build a model equivariant to $8$ rotations.
We indicate the group of $N$ discrete rotations as $C_N$, i.e. the **cyclic group** of order $N$.
In this case, we will use $C_8$.

Because the inputs are still gray-scale images, the input type of the model is again a *scalar field*.

However, internally we use *regular fields*: this is equivalent to a *group-equivariant convolutional neural network*.

Finally, we build *invariant* features for the final classification task by pooling over the group using *Group Pooling*.

The final classification is performed by a two fully connected layers.

# The model

Here is the definition of our model:

In [2]:
class C8SteerableCNN(nn.EquivariantModule):
    layers: list[eqx.Module]
    fully_net: list
    input_type: nn.FieldType

    def __init__(self, key, n_classes=10):
        super(C8SteerableCNN, self).__init__()

        keys = jax.random.split(key, 8)
        
        # the model is equivariant under rotations by 45 degrees, modelled by C8
        r2_act = gspaces.rot2dOnR2(N=8)
        
        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(r2_act, [r2_act.trivial_repr])
        
        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type
        
        # convolution 1
        # first specify the output type of the convolutional layer
        # we choose 24 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(r2_act, 24*[r2_act.regular_repr])
        block1 = nn.SequentialModule(
            nn.MaskModule(in_type, 29, margin=1),
            nn.R2Conv(in_type, out_type, kernel_size=7, padding=1, use_bias=False, key=keys[0]),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        )
        
        # convolution 2
        # the old output type is the input type to the next layer
        in_type = block1.out_type
        # the output type of the second convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 48*[r2_act.regular_repr])
        block2 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[1]),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        )
        pool1 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        
        # convolution 3
        # the old output type is the input type to the next layer
        in_type = block2.out_type
        # the output type of the third convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 48*[r2_act.regular_repr])
        block3 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[2]),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        )
        
        # convolution 4
        # the old output type is the input type to the next layer
        in_type = block3.out_type
        # the output type of the fourth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 96*[r2_act.regular_repr])
        block4 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[3]),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        )
        pool2 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        
        # convolution 5
        # the old output type is the input type to the next layer
        in_type = block4.out_type
        # the output type of the fifth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 96*[r2_act.regular_repr])
        block5 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[4]),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        )
        
        # convolution 6
        # the old output type is the input type to the next layer
        in_type = block5.out_type
        # the output type of the sixth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 64*[r2_act.regular_repr])
        block6 = nn.SequentialModule(
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=1, use_bias=False, key=keys[5]),
            nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        )
        pool3 = nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=1, padding=0)
        
        gpool = nn.GroupPooling(out_type)

        self.layers = [block1, block2, pool1, block3, block4, pool2, block5, block6, pool3, gpool]
        
        # number of output channels
        c = gpool.out_type.size
        
        # Fully Connected
        self.fully_net = [
            jnp.ravel,
            eqx.nn.Linear(c, 64, key=keys[6]),
            eqx.nn.BatchNorm(64, axis_name="batch"),
            jax.nn.elu,
            eqx.nn.Linear(64, n_classes, key=keys[7]),
        ]
    
    def __call__(self, input: Array, state: eqx.nn.State):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        x = nn.GeometricTensor(input, self.input_type)
        
        # apply each equivariant block
        for layer in self.layers:
            if "state" in inspect.signature(layer).parameters:
                x, state = layer(x, state)
            else:
                x = layer(x)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor
        
        # classify with the final fully connected layers)
        for layer in self.fully_net:
            if "state" in inspect.signature(layer).parameters:
                x, state = jax.vmap(layer, (0, None), (0, None), axis_name="batch")(x, state)
            else:
                x = jax.vmap(layer, axis_name="batch")(x)

        return x, state


Let's try the model on *rotated* MNIST

In [3]:
# download the dataset
!wget -nc http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip
# uncompress the zip file
!unzip -n mnist_rotation_new.zip -d mnist_rotation_new

/bin/bash: wget: command not found
Archive:  mnist_rotation_new.zip


In [3]:
from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import InterpolationMode

import numpy as np

from PIL import Image

Build the dataset

In [4]:
class MnistRotDataset(Dataset):
    
    def __init__(self, mode, transform=None):
        assert mode in ['train', 'test']
            
        if mode == "train":
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_train_valid.amat"
        else:
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_test.amat"
        
        self.transform = transform

        data = np.loadtxt(file, delimiter=' ')
            
        self.images = data[:, :-1].reshape(-1, 28, 28).astype(np.float32)
        self.labels = data[:, -1].astype(np.int64)
        self.num_samples = len(self.labels)
    
    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        image = Image.fromarray(image, mode='F')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.labels)

# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
pad = Pad((0, 0, 1, 1), fill=0)

# to reduce interpolation artifacts (e.g. when testing the model on rotated images),
# we upsample an image by a factor of 3, rotate it and finally downsample it again
resize1 = Resize(87)
resize2 = Resize(29)

totensor = ToTensor()

Let's build the model

In [5]:
key = jax.random.PRNGKey(0)
model = C8SteerableCNN(key)
state = eqx.nn.State(model)

The model is now randomly initialized. 
Therefore, we do not expect it to produce the right class probabilities.

However, the model should still produce the same output for rotated versions of the same image.
This is true for rotations by multiples of $\frac{\pi}{2}$, but is only approximate for rotations by $\frac{\pi}{4}$.

Let's test it on a random test image:
we feed eight rotated versions of the first image in the test set and print the output logits of the model for each of them.

In [6]:
def test_model(model: eqx.Module, x: Image):
    np.set_printoptions(linewidth=10000)
    
    # evaluate the `model` on 8 rotated versions of the input image `x`
    # model = model.eval()
    
    x = resize1(pad(x))
    
    print()
    print('##########################################################################################')
    header = 'angle |  ' + '  '.join(["{:6d}".format(d) for d in range(10)])
    print(header)
    # with torch.no_grad():
    for r in range(8):
        x_transformed = totensor(resize2(x.rotate(r*45., Image.BILINEAR))).reshape(1, 1, 29, 29)
        # print("x_transformed", type(x_transformed), x_transformed)
        x_transformed = jnp.array(x_transformed.numpy())#.to(device)

        y, _ = model(x_transformed, state)
        # y = y.to('cpu').numpy().squeeze()
        y = np.array(y).squeeze()
        
        angle = r * 45
        print("{:5d} : {}".format(angle, y))
    print('##########################################################################################')
    print()

In [7]:
# build the test set    
raw_mnist_test = MnistRotDataset(mode='test')

In [8]:
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))

# evaluate the model
test_model(model, x)


##########################################################################################
angle |       0       1       2       3       4       5       6       7       8       9
    0 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
   45 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
   90 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  135 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  180 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  225 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  270 : [-0.11777765  0.00718439 -0.0608

The output of the model is already almost invariant.
However, we still observe small fluctuations in the outputs.

This is because the model contains some operations which might break equivariance.
For instance, every convolution includes a padding of $2$ pixels per side. This is adds information about the actual orientation of the grid where the image/feature map is sampled because the padding is not rotated with the image. 

During training, the model will observe rotated patterns and will learn to ignore the noise coming from the padding.

So, let's train the model now.
The model is exactly the same used to train a normal *PyTorch* architecture:

In [9]:
train_transform = Compose([
    pad,
    resize1,
    RandomRotation(180., interpolation=InterpolationMode.BILINEAR, expand=False),
    resize2,
    totensor,
])

mnist_train = MnistRotDataset(mode='train', transform=train_transform)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64)


test_transform = Compose([
    pad,
    totensor,
])
mnist_test = MnistRotDataset(mode='test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64)

In [12]:

@eqx.filter_jit
def loss(
    model: nn.EquivariantModule, state: eqx.nn.State, x: Float[Array, "batch 1 29 29"], y: Int[Array, " batch"]
) -> Float[Array, ""]:
    # Our input has the shape (BATCH_SIZE, 1, 29, 29), but our model operations on
    # a single input input image of shape (1, 29, 29).
    pred_y, state = model(x, state)
    loss = cross_entropy(y, pred_y)
    return loss, state


@eqx.filter_jit
def loss_wrapper(params, static, *args):
    is_param = lambda x: isinstance(x, nn.ParameterArray)
    model = eqx.combine(params, static, is_leaf=is_param)
    return loss(model, *args)


def cross_entropy(
    y: Int[Array, " batch"], pred_y: Float[Array, "batch 10"]
) -> Float[Array, ""]:
    # y are the true targets, and should be integers 0-9.
    # pred_y are the logits predictions.
    return jnp.mean(optax.softmax_cross_entropy(logits=pred_y, labels=jax.nn.one_hot(y, 10)))


@eqx.filter_jit
def compute_accuracy(
    model: nn.EquivariantModule, x: Float[Array, "batch 1 29 29"], y: Int[Array, " batch"]
) -> Float[Array, ""]:
    """This function takes as input the current model
    and computes the average accuracy on a batch.
    """
    pred_y, _ = model(x)
    pred_y = jnp.argmax(pred_y, axis=1)
    return jnp.mean(y == pred_y)

def evaluate(model: nn.EquivariantModule, state: eqx.nn.State, test_loader: torch.utils.data.DataLoader):
    """This function evaluates the model on the test dataset,
    computing both the average loss and the average accuracy.
    """
    inference_model = model.eval()
    inference_model = eqx.Partial(inference_model, state=state)

    avg_acc = 0
    for x, y in test_loader:
        x = jnp.array(x.numpy())
        y = jnp.array(y.numpy())
        # Note that all the JAX operations happen inside `loss` and `compute_accuracy`,
        # and both have JIT wrappers, so this is fast.
        avg_acc += compute_accuracy(inference_model, x, y)
    return avg_acc / len(test_loader)


def train(
    model: eqx.Module,
    state: eqx.nn.State,
    train_loader: torch.utils.data.DataLoader,
    test_loader: torch.utils.data.DataLoader,
    optim: optax.GradientTransformation,
    epochs: int,
    print_every: int,
) -> eqx.Module:
    # Just like earlier: It only makes sense to train the arrays in our model,
    # so filter out everything else.
    is_param = lambda x: isinstance(x, nn.ParameterArray)
    opt_state = optim.init(eqx.filter(model, is_param, is_leaf=is_param))

    # Always wrap everything -- computing gradients, running the optimiser, updating
    # the model -- into a single JIT region. This ensures things run as fast as
    # possible.
    @eqx.filter_jit
    def make_step(
        model: eqx.Module,
        state: eqx.nn.State, 
        opt_state: PyTree,
        x: Float[Array, "batch 1 29 29"],
        y: Int[Array, " batch"],
    ):
        params, static = eqx.partition(model, is_param, is_leaf=is_param)
        (loss_value, state), grads = jax.value_and_grad(loss_wrapper, has_aux=True)(params, static, state, x, y)
        updates, opt_state = optim.update(grads, opt_state, model)
        model = eqx.apply_updates(model, updates)
        return model, state, opt_state, loss_value

    for epoch in range(epochs):

        for step, (x, y) in enumerate(train_loader):
            # PyTorch dataloaders give PyTorch tensors by default,
            # so convert them to NumPy arrays.
            x = jnp.array(x.numpy())
            y = jnp.array(y.numpy())
            model, state, opt_state, train_loss = make_step(model, state, opt_state, x, y)

        if (epoch % print_every) == 0 or (epoch == epochs - 1):
            test_accuracy = evaluate(model, state, test_loader)
            print(
                f"{epoch=}",
                f"{step=}, train_loss={train_loss.item():.2f}, "
                f"test_accuracy={100*test_accuracy.item():.1f}%"
            )
    return model


In [13]:
optim = optax.adamw(learning_rate=5e-5, weight_decay=1e-5)

steps = 31
print_every = 5
model = train(model, state, train_loader, test_loader, optim, steps, print_every)

epoch=0 step=187, train_loss=0.6, test_accuracy=0.9
epoch=5 step=187, train_loss=0.7, test_accuracy=0.9
epoch=10 step=187, train_loss=0.7, test_accuracy=1.0
epoch=15 step=187, train_loss=0.7, test_accuracy=1.0
epoch=20 step=187, train_loss=0.7, test_accuracy=1.0
epoch=25 step=187, train_loss=0.7, test_accuracy=1.0
epoch=30 step=187, train_loss=0.7, test_accuracy=1.0


In [14]:
    
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))


# evaluate the model
test_model(model, x)


##########################################################################################
angle |       0       1       2       3       4       5       6       7       8       9
    0 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
   45 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
   90 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  135 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  180 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  225 : [-0.11777765  0.00718439 -0.06087103 -0.00774419 -0.07453537  0.05387858 -0.03126591 -0.00302693 -0.06451857 -0.07264927]
  270 : [-0.11777765  0.00718439 -0.0608